In [2]:
import os
import re
import csv

def load_sql_keywords(file_path):
    """
    Loads SQL keywords from a text file into a set for fast lookup.
    
    Parameters:
    - file_path: Path to the sql_keywords.txt file.
    
    Returns:
    - A set containing all SQL keywords in lowercase.
    """
    sql_keywords = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            sql_keywords.add(line.strip().lower())  # Convert to lowercase for consistency
    return sql_keywords

def extract_words_from_schema(file_path, sql_keywords, unique_words):
    """
    Extracts words from schema.sql that are not in the SQL keywords list.

    Parameters:
    - file_path: Path to the schema.sql file.
    - sql_keywords: A set of SQL keywords to be ignored.
    - unique_words: A set to store unique non-SQL words.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        # Extract all words consisting of letters and underscores, excluding numbers
        words = re.findall(r'\b[a-zA-Z_]+\b', content)
        for word in words:
            if word.lower() not in sql_keywords:  # Ignore SQL keywords
                unique_words.add(word)

def extract_all_schema_words(database_folder, sql_keywords_file, output_csv):
    """
    Extracts words from all schema.sql files in the database folder that are not SQL keywords.
    
    Parameters:
    - database_folder: The path to the folder containing subfolders with schema.sql files.
    - sql_keywords_file: Path to the sql_keywords.txt file.
    - output_csv: Path to the CSV file where the extracted words will be saved.
    """
    sql_keywords = load_sql_keywords(sql_keywords_file)
    unique_words = set()

    # Walk through all folders in the database directory
    for root, dirs, files in os.walk(database_folder):
        for file in files:
            if file == 'schema.sql':  # Check if the file is schema.sql
                schema_path = os.path.join(root, file)
                extract_words_from_schema(schema_path, sql_keywords, unique_words)

    # Save the unique words to the CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['unique_words'])  # Header row
        for word in sorted(unique_words):
            writer.writerow([word])

    print(f"Extraction completed. Output saved as: {output_csv}")

def main():
    """
    Main function to execute the extraction process.
    Defines input and output paths and ensures the output directory exists.
    """
    # Paths to the directories and files
    # script_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of the current script
    database_folder = '../data/spider/database' # Folder with schema.sql files
    sql_keywords_file = '../data/sql_keywords.txt'  # SQL keywords file
    output_csv = '../data/extracted/extracted_all_schema_words.csv' # Output CSV file

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)

    # Run the extraction process
    extract_all_schema_words(database_folder, sql_keywords_file, output_csv)

main()

Extraction completed. Output saved as: ../data/extracted/extracted_all_schema_words.csv
